In [8]:
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
import pandas as pd
from collections import OrderedDict
from datetime import date
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import fiona
import shapely
import datetime

In [9]:
"""
AOI Parameters
"""

# bbox
bbox = (-4.860230,47.428380,-0.982056,48.897512)
bbox_poly = shapely.geometry.box(*bbox, ccw=True)


# start-end times
start = datetime.datetime(2018,4,1)
end = datetime.datetime(2018,9,30)

In [10]:
# define tiles
tiles = ['30UUU', '30UVV', '30UVU', '30TVT', '30UWU', '30TWT', '30UXU']


# set up API
user = "sdonike" ####### ## change this!
pw =  "JH9EZFHhfySLRqTvZR8q" ######## ## change this!
api = SentinelAPI(user, pw, api_url='https://scihub.copernicus.eu/dhus', show_progressbars= True,timeout=200)


In [58]:
def get_footprints(tiles,lower_cloud,upper_cloud,producttype,processinglevel):
    # set query params
    query_kwargs = {
        'platformname': 'Sentinel-2',
        #S2B_MSIL2A_20180929T110829_N0208_R137_T30TWT_20180929T155623,
        'producttype': producttype,
        #'filename' : 'S2B_MSIL1C_20211105T112149_N0301_R037_T30UWU_20211108T162544.SAFE',
        #'processinglevel' : processinglevel,
        'date': ('20180401', '20180930'),
        'cloudcoverpercentage': (lower_cloud, upper_cloud),
        # 'limit': 999
    } 
    
    #products_ls = []
    footprints = []
    titles = []
    products = OrderedDict()
    for tile in tiles:
        products = OrderedDict()
        kw = query_kwargs.copy()
        kw['tileid'] = tile  
        pp = api.query(**kw)
        products.update(pp)
        #products_ls.append(products)
        print("Min. CC:",lower_cloud,"- Max. CC",upper_cloud,"\tTile:",tile,"\t - Products:",len(products))


        products_df = api.to_dataframe(products)
        #products_df_sorted = products_df.sort_values(['size'],ascending = [False])
        #products_df_sorted = products_df_sorted.head (1)

        #print(products_df_sorted['size'])
        #print(products_df_sorted['title'])

        fp = []
        titles_tmp = []
        for i in products_df["footprint"]:
            fp.append(i)
        for i in products_df["title"]:
            titles_tmp.append(str(i))
            #print(i)

        #tmp_ls = [products_df_sorted["title"],products_df_sorted["tileid"],products_df_sorted["footprint"],products_df_sorted["datatakesensingstart"]]

        footprints.append(fp)
        titles.append(titles_tmp)


        #download data
        # api.download_all(products_df_sorted.index, directory_path= r'C:\Users\EQuilibrium\PyBegining\downloader\Downld_Sen',max_attempts=2, checksum = True )
        #api.download(products_df_sorted.index, directory_path= r'C:\Users\EQuilibrium\PyBegining\downloader\Downld_Sen', checksum = True )
        #print(products_df_sorted)
    
    # flatten list
    footprints = [item for sublist in footprints for item in sublist]
    titles = [item for sublist in titles for item in sublist]
    footprints_df = pd.DataFrame(footprints,columns=["geometry"])
    
    titles_df = pd.DataFrame(titles,columns=["titles"])
    #footprints_df
    
    footprints_df['geometry'] = gpd.GeoSeries.from_wkt(footprints_df['geometry'])
    gdf = gpd.GeoDataFrame(footprints_df, geometry='geometry')
    gdf["titles"] = titles

    return(gdf)

In [59]:
a = get_footprints(tiles,0, 5,"S2MSI1C","Level-1C")#.to_file('qgis_files/sen2_tiles/footprints_05.shp')
#get_footprints(tiles,0,50,"S2MSI1C","Level-1C").to_file('qgis_files/sen2_tiles/footprints_10.shp')  
#get_footprints(tiles,10,15,"S2MSI1C","Level-1C").to_file('qgis_files/sen2_tiles/footprints_15.shp')  
#get_footprints(tiles,15,20,"S2MSI1C","Level-1C").to_file('qgis_files/sen2_tiles/footprints_20.shp')  

Min. CC: 0 - Max. CC 5 	Tile: 30UUU 	 - Products: 36
Min. CC: 0 - Max. CC 5 	Tile: 30UVV 	 - Products: 13
Min. CC: 0 - Max. CC 5 	Tile: 30UVU 	 - Products: 12
Min. CC: 0 - Max. CC 5 	Tile: 30TVT 	 - Products: 21
Min. CC: 0 - Max. CC 5 	Tile: 30UWU 	 - Products: 28
Min. CC: 0 - Max. CC 5 	Tile: 30TWT 	 - Products: 57
Min. CC: 0 - Max. CC 5 	Tile: 30UXU 	 - Products: 16


In [62]:
a["titles"][0]

'S2B_MSIL1C_20180929T110829_N0206_R137_T30UUU_20180929T150202'

In [115]:
def get_tiles(titles):
    for title in titles:
        
        #title = title.replace("L1C","L2A") +".SAFE"
        title = title+".SAFE"
        print(title)
        query_kwargs = {
        #'platformname': 'Sentinel-2',
        #'producttype': producttype,
        'filename' : title,
        'limit': 999
        }
        
        
        products = OrderedDict()
        kw = query_kwargs.copy()
        pp = api.query(**query_kwargs)
        products.update(pp)
        products_df = api.to_dataframe(products)
        #api.download_all(products)
        print(products_df)

In [116]:
get_tiles(a["titles"])

S2B_MSIL1C_20180929T110829_N0206_R137_T30UUU_20180929T150202.SAFE
                                                                                  title  \
65f3bdc2-ae92-4fbf-8303-aa9ee589a5c6  S2B_MSIL1C_20180929T110829_N0206_R137_T30UUU_2...   

                                                                                   link  \
65f3bdc2-ae92-4fbf-8303-aa9ee589a5c6  https://scihub.copernicus.eu/dhus/odata/v1/Pro...   

                                                                       link_alternative  \
65f3bdc2-ae92-4fbf-8303-aa9ee589a5c6  https://scihub.copernicus.eu/dhus/odata/v1/Pro...   

                                                                              link_icon  \
65f3bdc2-ae92-4fbf-8303-aa9ee589a5c6  https://scihub.copernicus.eu/dhus/odata/v1/Pro...   

                                                                                summary  \
65f3bdc2-ae92-4fbf-8303-aa9ee589a5c6  Date: 2018-09-29T11:08:29.024Z, Instrument: MS...   

                   

KeyboardInterrupt: 

In [ ]:
def get_tiles(tiles,lower_cloud,upper_cloud,producttype,processinglevel):
    # set query params
    query_kwargs = {
        'platformname': 'Sentinel-2',
        #S2B_MSIL2A_20180929T110829_N0208_R137_T30TWT_20180929T155623,
        'producttype': producttype,
        #'filename' : 'S2B_MSIL1C_20211105T112149_N0301_R037_T30UWU_20211108T162544.SAFE',
        #'processinglevel' : processinglevel,
        'date': ('20180401', '20180930'),
        'cloudcoverpercentage': (lower_cloud, upper_cloud),
        # 'limit': 999
    } 
    
    #products_ls = []
    footprints = []
    titles = []
    products = OrderedDict()
    for title in titles:
        products = OrderedDict()
        kw = query_kwargs.copy()
        kw['tileid'] = tile  
        pp = api.query(**kw)
        products.update(pp)
        #products_ls.append(products)
        print("Min. CC:",lower_cloud,"- Max. CC",upper_cloud,"\tTile:",tile,"\t - Products:",len(products))


        products_df = api.to_dataframe(products)
        #products_df_sorted = products_df.sort_values(['size'],ascending = [False])
        #products_df_sorted = products_df_sorted.head (1)

        #print(products_df_sorted['size'])
        #print(products_df_sorted['title'])

        fp = []
        titles_tmp = []
        for i in products_df["footprint"]:
            fp.append(i)
        for i in products_df["title"]:
            titles_tmp.append(str(i))
            #print(i)

        #tmp_ls = [products_df_sorted["title"],products_df_sorted["tileid"],products_df_sorted["footprint"],products_df_sorted["datatakesensingstart"]]

        footprints.append(fp)
        titles.append(titles_tmp)


        #download data
        # api.download_all(products_df_sorted.index, directory_path= r'C:\Users\EQuilibrium\PyBegining\downloader\Downld_Sen',max_attempts=2, checksum = True )
        #api.download(products_df_sorted.index, directory_path= r'C:\Users\EQuilibrium\PyBegining\downloader\Downld_Sen', checksum = True )
        #print(products_df_sorted)
    
    # flatten list
    footprints = [item for sublist in footprints for item in sublist]
    titles = [item for sublist in titles for item in sublist]
    footprints_df = pd.DataFrame(footprints,columns=["geometry"])
    
    titles_df = pd.DataFrame(titles,columns=["titles"])
    #footprints_df
    
    footprints_df['geometry'] = gpd.GeoSeries.from_wkt(footprints_df['geometry'])
    gdf = gpd.GeoDataFrame(footprints_df, geometry='geometry')
    gdf["titles"] = titles

    return(gdf)